<a href="https://colab.research.google.com/github/vaibhavvesmaker/Spotfire/blob/main/Spotifier_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 7.5 MB/s eta 0:00:00


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set your Spotify API credentials
client_id = 'bbdc86e8622e4543bf8ae1ae18c4e614'
client_secret = '0b3cf48a9c6d4f81861cafe3b386a295'

# Authenticate with the Spotify API
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Load Spotify dataset
spotify_data = pd.read_csv(r"/content/drive/My Drive/spotify_data.csv")

# Split dataset into features (X) and target variable (Y)
X = spotify_data.drop(['target', 'Unnamed: 0', 'song_title', 'artist'], axis=1)
Y = spotify_data['target']

# Split the data into training and testing sets
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=seed)

# Train Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)


# Function to get relevant features of a song by its Spotify name
def get_song_features_by_name(song_name):
    results = sp.search(q=song_name, type='track', limit=1)

    if not results['tracks']['items']:
        print(f"No results found for the song '{song_name}'.")
        return None

    track_uri = results['tracks']['items'][0]['uri']
    features = sp.audio_features(track_uri)


    if not features:
        print(f"No audio features found for the song '{song_name}'.")
        return None

    relevant_features = [
        features[0]['acousticness'],
        features[0]['danceability'],
        features[0]['duration_ms'],
        features[0]['energy'],
        features[0]['instrumentalness'],
        features[0]['key'],
        features[0]['liveness'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['valence']
    ]

    return relevant_features

# Take user input for a song name (case-insensitive)
song_name = input("Enter the name of the song: ").lower()

# Get features for the user's song
user_song_features = get_song_features_by_name(song_name)

if user_song_features:
    # Set feature names on the input data
    user_song_data = pd.DataFrame([user_song_features], columns=X.columns)

    print(user_song_data)

    # Make prediction for the user's song
    user_prediction = rf_model.predict(user_song_data)

    # Display the prediction
    if user_prediction[0] == 1:
        print("You might like this song!")
    else:
        print("You might not like this song.")


Enter the name of the song: perfect
   acousticness  danceability  duration_ms  energy  instrumentalness  key  \
0        0.0248         0.667       161250   0.845             0.065    6   

   liveness  loudness  mode  speechiness    tempo  time_signature  valence  
0     0.466    -6.845     1       0.0812  127.879               4    0.681  
You might like this song!
